### Removing Duplicates

### Handling missing values

### Feature Selection
For now we are not interested in some columns, like for example all hyperlinks (since most of them are expired anyways), as well as all columns that don't have any values like county, as well as columns that don't pose a functional value for data analysis
For our further data exploration we decide to remove these columns to get a better overview of the data

(Keep in mind that we might be able to extract the region from the url, should the dedicated region feature show problems)

In [ ]:
df = df.drop(['image_url', 'county', 'region_url', 'url', 'id'], axis=1)